In [244]:
# Import packages
import geopandas as gpd
import numpy as np
import pandas as pd
import libpysal
import networkx as nx
import osmnx as ox
import time
from shapely import geometry
from shapely.geometry import Point, MultiLineString
from shapely.ops import nearest_points
import matplotlib.pyplot as plt
from itertools import product
import multiprocessing as mp
import math

In [245]:
start = time.time()
# import nodes
road_node = gpd.read_file(r'D:\Dumps\All_2km buffer\Philadelphia 2km buffer_all\nodes.shp')
road_node = road_node.set_index('osmid')
road_nodes = road_node.to_crs(4326)
road_nodes['geometry_m'] = gpd.GeoSeries(road_nodes['geometry'], crs = 4326).to_crs(3043)
road_nodes

,y,x,street_cou,lon,lat,highway,ref,geometry,geometry_m
osmid,,,,,,,,,
103237949,4.431781e+06,503033.785157,3,-74.964440,40.036252,None,None,POINT (-74.96444 40.03625),POINT (-5689489.546 8448457.709)
103353127,4.431724e+06,502937.820011,3,-74.965565,40.035733,None,None,POINT (-74.96557 40.03573),POINT (-5689617.107 8448567.833)
103353090,4.431839e+06,503124.706820,3,-74.963374,40.036770,None,None,POINT (-74.96337 40.03677),POINT (-5689364.441 8448354.775)
103237976,4.431638e+06,503122.315306,4,-74.963403,40.034959,None,None,POINT (-74.96340 40.03496),POINT (-5689653.040 8448263.511)
103238007,4.431510e+06,503203.675365,4,-74.962450,40.033810,None,None,POINT (-74.96245 40.03381),POINT (-5689797.056 8448087.089)
...,...,...,...,...,...,...,...,...,...
9703813717,4.416438e+06,483689.901846,3,-75.190791,39.897861,None,None,POINT (-75.19079 39.89786),POINT (-5720575.973 8468970.364)
9703813743,4.416385e+06,483760.410069,3,-75.189965,39.897385,None,None,POINT (-75.18996 39.89738),POINT (-5720619.056 8468844.195)
9703813748,4.416433e+06,484049.808886,3,-75.186581,39.897820,None,None,POINT (-75.18658 39.89782),POINT (-5720414.758 8468450.854)


In [246]:
# import edges
road_edge = gpd.read_file(r'D:\Dumps\All_2km buffer\Philadelphia 2km buffer_all\edges.shp')
road_edge = road_edge.set_index(['u','v','key'])
road_edges = road_edge.to_crs(4326)
road_edges['geometry_m'] = gpd.GeoSeries(road_edges['geometry'], crs = 4326).to_crs(3043)
road_edges

#Get both ways
RE = road_edges.reset_index()
RE2 = RE[['osmid','from','to','name','length','geometry']]
RE2.columns = ['osmid','to','from','name','length','geometry']
RE2 = RE2[['osmid','from','to','name','length','geometry']]
road_connections = pd.concat([RE, RE2]).to_crs(4326)
road_connections['key'] = road_connections['from'].astype(str) + '-' + road_connections['to'].astype(str)

In [247]:
# Get the graph object
graph = ox.graph_from_place('Philadelphia, United States', network_type="all", buffer_dist = 2000)
graph = ox.speed.add_edge_speeds(graph)
graph = ox.speed.add_edge_travel_times(graph)

In [248]:
# Import greenspace and population grids
Penn_bound = gpd.read_file(r'C:\Users\bartb\Downloads\Pennsylvania\boundaries.shp')
Philly_greenspace = gpd.read_file(r'C:\Users\bartb\Downloads\OneDrive_1_03-05-2022\Public_greenspace.gpkg')
Philly_popgrid = gpd.read_file(r'C:\Users\bartb\Downloads\OneDrive_1_03-05-2022\CPoPGrid.gpkg')
Philly_bound = Penn_bound[Penn_bound['municipal1'] == 'PHILADELPHIA']

In [249]:
# Set crs to WGS84
Philly_bound = Philly_bound.to_crs(4326)
Philly_greenspace = Philly_greenspace.to_crs(4326)
Philly_popgrid = Philly_popgrid.to_crs(4326)

In [250]:
# Make sure only complete overlays progress and take their centroids
popgrid = Philly_popgrid.overlay(Philly_bound)
popgrid['centroid'] = popgrid.to_crs(4326).centroid
popgrid['centroid_m'] = popgrid['centroid'].to_crs(3043)
popgrid['area'] = popgrid.area / popgrid.area.max()
popgrid = popgrid[(popgrid['area'] >= 0.99) & popgrid['PoP2015_Number'] > 0]
popgrid['grid_lon'] = popgrid['centroid_m'].x
popgrid['grid_lat'] = popgrid['centroid_m'].y
popgrid = popgrid.reset_index()
popgrid

,index,grid_id,PoP2015_Number,class_of_m,county,fed_aid_ur,fed_id_num,fips_area,fips_count,fips_mun_c,...,gpid,mslink,municipal,municipal1,geometry,centroid,centroid_m,area,grid_lon,grid_lat
0,6,259,113,CITY,67,4,23-6003047,00004,101,60000,...,39973.0,2573.0,301,PHILADELPHIA,"POLYGON ((-75.01452 40.13521, -75.01452 40.133...",POINT (-75.01564 40.13435),POINT (-5675978.406 8459812.552),0.996208,-5.675978e+06,8.459813e+06
1,7,260,135,CITY,67,4,23-6003047,00004,101,60000,...,39973.0,2573.0,301,PHILADELPHIA,"POLYGON ((-75.01227 40.13521, -75.01227 40.133...",POINT (-75.01339 40.13435),POINT (-5675888.831 8459539.227),0.996208,-5.675889e+06,8.459539e+06
2,13,400,137,CITY,67,4,23-6003047,00004,101,60000,...,39973.0,2573.0,301,PHILADELPHIA,"POLYGON ((-75.01676 40.13349, -75.01676 40.131...",POINT (-75.01789 40.13262),POINT (-5676341.311 8459996.328),0.996234,-5.676341e+06,8.459996e+06
3,16,403,109,CITY,67,4,23-6003047,00004,101,60000,...,39973.0,2573.0,301,PHILADELPHIA,"POLYGON ((-75.01003 40.13349, -75.01003 40.131...",POINT (-75.01115 40.13262),POINT (-5676072.561 8459176.309),0.996234,-5.676073e+06,8.459176e+06
4,17,404,109,CITY,67,4,23-6003047,00004,101,60000,...,39973.0,2573.0,301,PHILADELPHIA,"POLYGON ((-75.00778 40.13349, -75.00778 40.131...",POINT (-75.00890 40.13262),POINT (-5675982.949 8458902.986),0.996234,-5.675983e+06,8.458903e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3886,10223,20883,5,CITY,67,4,23-6003047,00004,101,60000,...,39973.0,2573.0,301,PHILADELPHIA,"POLYGON ((-75.25706 39.88308, -75.25706 39.881...",POINT (-75.25819 39.88221),POINT (-5725754.488 8476450.010),0.999924,-5.725754e+06,8.476450e+06
3887,10225,20885,39,CITY,67,4,23-6003047,00004,101,60000,...,39973.0,2573.0,301,PHILADELPHIA,"POLYGON ((-75.25257 39.88308, -75.25257 39.881...",POINT (-75.25369 39.88221),POINT (-5725576.204 8475897.317),0.999924,-5.725576e+06,8.475897e+06
3888,10226,20886,5,CITY,67,4,23-6003047,00004,101,60000,...,39973.0,2573.0,301,PHILADELPHIA,"POLYGON ((-75.25033 39.88308, -75.25033 39.881...",POINT (-75.25145 39.88221),POINT (-5725487.039 8475620.982),0.999924,-5.725487e+06,8.475621e+06
3889,10275,21024,3,CITY,67,4,23-6003047,00004,101,60000,...,39973.0,2573.0,301,PHILADELPHIA,"POLYGON ((-75.25931 39.88135, -75.25931 39.879...",POINT (-75.26043 39.88048),POINT (-5726119.978 8476637.252),0.999949,-5.726120e+06,8.476637e+06


In [252]:
green_buffer = gpd.GeoDataFrame(geometry = Philly_greenspace.to_crs(3043).buffer(25).to_crs(4326))
Philly_greenspace['geometry_w_buffer'] = green_buffer
Philly_greenspace['geometry_w_buffer'] = gpd.GeoSeries(Philly_greenspace['geometry_w_buffer'])
Philly_greenspace['geom buffer diff'] = Philly_greenspace['geometry_w_buffer'].difference(Philly_greenspace['geometry'])

In [253]:
# This function group components in itself that overlap (with the buffer set of 25 metres)
# https://stackoverflow.com/questions/68036051/geopandas-self-intersection-grouping
W = libpysal.weights.fuzzy_contiguity(Philly_greenspace['geometry_w_buffer'])
Philly_greenspace['components'] = W.component_labels
Philly_parks = Philly_greenspace.dissolve('components')

# Exclude parks below 0.04 ha.
Philly_parks = Philly_parks[Philly_parks.to_crs(3043).area > 400]
Philly_parks

,geometry,OBJECTID,FeatClass,Category,Own_Type,Own_Name,Loc_Own,Mang_Type,Mang_Name,Loc_Mang,...,IUCNCtSrc,IUCNCtDt,Date_Est,Comments,EsmtHldr,EHoldTyp,SHAPE_Leng,SHAPE_Area,geometry_w_buffer,geom buffer diff
components,,,,,,,,,,,,,,,,,,,,,
0,"POLYGON ((-75.09099 39.91406, -75.09099 39.914...",74457,Fee,Fee,LOC,CNTY,Camden County,LOC,CNTY,Camden County,...,GAP - Default,2020,None,None,None,None,4153.613200,2.281885e+05,"POLYGON ((-75.09191 39.91345, -75.09192 39.913...","POLYGON ((-75.09192 39.91345, -75.09193 39.913..."
1,"MULTIPOLYGON (((-75.02442 39.89516, -75.02516 ...",74458,Fee,Fee,LOC,CNTY,Camden County,LOC,CNTY,Camden County,...,GAP - Default,2020,None,None,None,None,2838.997916,5.667747e+04,"POLYGON ((-75.02347 39.89392, -75.02348 39.893...","POLYGON ((-75.02348 39.89391, -75.02350 39.893..."
2,"MULTIPOLYGON (((-75.09305 39.93204, -75.09310 ...",74458,Fee,Fee,LOC,CNTY,Camden County,LOC,CNTY,Camden County,...,GAP - Default,2020,None,None,None,None,2838.997916,5.667747e+04,"POLYGON ((-75.08855 39.93442, -75.08855 39.934...","POLYGON ((-75.08855 39.93442, -75.08859 39.934..."
3,"MULTIPOLYGON (((-75.06332 39.87934, -75.06355 ...",74462,Fee,Fee,LOC,CNTY,Camden County,LOC,CNTY,Camden County,...,GAP - Default,2020,None,None,None,None,3919.908080,1.591219e+05,"POLYGON ((-75.07984 39.88138, -75.07984 39.881...","POLYGON ((-75.07984 39.88137, -75.07983 39.881..."
4,"POLYGON ((-75.07444 39.89581, -75.07452 39.895...",74462,Fee,Fee,LOC,CNTY,Camden County,LOC,CNTY,Camden County,...,GAP - Default,2020,None,None,None,None,3919.908080,1.591219e+05,"POLYGON ((-75.07434 39.89568, -75.07442 39.895...","POLYGON ((-75.07442 39.89565, -75.07444 39.895..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
742,"POLYGON ((-75.18593 40.13635, -75.18627 40.135...",242041,Fee,Fee,LOC,CITY,Upper Dublin Township,LOC,CITY,Upper Dublin Township,...,GAP - Default,2020,None,None,None,None,1490.477107,6.204571e+04,"POLYGON ((-75.18593 40.13650, -75.18592 40.136...","POLYGON ((-75.18592 40.13650, -75.18590 40.136..."
743,"POLYGON ((-75.26429 40.09641, -75.26362 40.096...",244970,Fee,Fee,LOC,CITY,WHITEMARSH TOWNSHIP,LOC,CITY,WHITEMARSH TOWNSHIP,...,GAP - Default,2020,None,None,None,None,1275.860609,8.335866e+04,"POLYGON ((-75.26442 40.09652, -75.26440 40.096...","POLYGON ((-75.26440 40.09653, -75.26438 40.096..."
744,"POLYGON ((-75.20583 40.11865, -75.20772 40.116...",244971,Fee,Fee,LOC,CITY,WHITEMARSH TOWNSHIP,LOC,CITY,WHITEMARSH TOWNSHIP,...,GAP - Default,2020,None,None,None,None,756.792370,2.828873e+04,"POLYGON ((-75.20571 40.11877, -75.20569 40.118...","POLYGON ((-75.20569 40.11876, -75.20568 40.118..."


In [254]:
# Take a buffer for the parks to ensure equal measurements for edge population grids
boundaries_buffer = gpd.GeoDataFrame(geometry = Philly_bound.to_crs(3043).buffer(1000).to_crs(4326))
parks_within_range = Philly_parks.overlay(boundaries_buffer)
parks_within_range['centroid'] = parks_within_range.centroid
parks_within_range['centroid_m'] = parks_within_range['centroid'].to_crs(3043)
parks_within_range['geometry_w_buffer'] = gpd.GeoSeries(parks_within_range['geometry_w_buffer'], crs = 4326)
parks_within_range['geometry_w_buffer_m'] = parks_within_range['geometry_w_buffer'].to_crs(3043)
parks_within_range

,OBJECTID,FeatClass,Category,Own_Type,Own_Name,Loc_Own,Mang_Type,Mang_Name,Loc_Mang,Des_Tp,...,EsmtHldr,EHoldTyp,SHAPE_Leng,SHAPE_Area,geometry_w_buffer,geom buffer diff,geometry,centroid,centroid_m,geometry_w_buffer_m
0,81288,Fee,Fee,LOC,CNTY,Montgomery County,LOC,CNTY,Montgomery County,LOTH,...,Natural Lands Trust,NGO,2167.370365,1.070843e+05,"POLYGON ((-75.25609 40.06118, -75.25608 40.061...","POLYGON ((-75.25608 40.06117, -75.25606 40.061...","MULTIPOLYGON (((-75.24324 40.05599, -75.24326 ...",POINT (-75.25165 40.05653),POINT (-5697693.589 8484600.730),"POLYGON ((-5697129.449 8485382.576, -5697129.4..."
1,82731,Fee,Fee,LOC,CNTY,Philadelphia County,LOC,CNTY,Philadelphia County,LOTH,...,None,None,36128.050943,3.353065e+06,"POLYGON ((-75.24691 39.91683, -75.24696 39.916...","POLYGON ((-75.24696 39.91680, -75.24697 39.916...","POLYGON ((-75.24706 39.91693, -75.24708 39.916...",POINT (-75.24714 39.91942),POINT (-5719375.750 8477008.358),"POLYGON ((-5719780.450 8476846.530, -5719785.6..."
2,82731,Fee,Fee,LOC,CNTY,Philadelphia County,LOC,CNTY,Philadelphia County,LOTH,...,None,None,36128.050943,3.353065e+06,"POLYGON ((-75.24828 39.92126, -75.24836 39.921...","POLYGON ((-75.24836 39.92121, -75.24837 39.921...","MULTIPOLYGON (((-75.23911 39.94226, -75.23909 ...",POINT (-75.25640 39.96241),POINT (-5712881.953 8480355.991),"POLYGON ((-5719127.059 8477242.926, -5719138.2..."
3,1260,Fee,Fee,FED,NPS,NPS,FED,NPS,NPS,HCA,...,None,None,216.240707,2.114309e+03,"POLYGON ((-75.15037 39.96185, -75.15037 39.961...","POLYGON ((-75.15037 39.96186, -75.15038 39.961...","POLYGON ((-75.14993 39.96159, -75.15020 39.961...",POINT (-75.15001 39.96195),POINT (-5708732.995 8467282.252),"POLYGON ((-5708763.625 8467321.486, -5708762.4..."
4,1348,Fee,Fee,FED,NPS,NPS,FED,NPS,NPS,HCA,...,None,None,6162.483495,1.285207e+05,"POLYGON ((-75.17280 40.03368, -75.17280 40.033...","POLYGON ((-75.17280 40.03369, -75.17280 40.033...","MULTIPOLYGON (((-75.17209 40.03406, -75.17261 ...",POINT (-75.17237 40.03387),POINT (-5698171.846 8473736.251),"POLYGON ((-5698218.752 8473779.070, -5698217.1..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
469,242021,Fee,Fee,LOC,CITY,Upper Darby Township,LOC,CITY,Upper Darby Township,LP,...,None,None,217.559826,1.590417e+03,"POLYGON ((-75.25744 39.94930, -75.25744 39.949...","POLYGON ((-75.25744 39.94930, -75.25806 39.949...","POLYGON ((-75.25807 39.94941, -75.25801 39.949...",POINT (-75.25763 39.94953),POINT (-5714985.845 8479845.686),"POLYGON ((-5715014.449 8479809.842, -5715014.8..."
470,242022,Fee,Fee,LOC,CITY,Upper Darby Township,LOC,CITY,Upper Darby Township,LP,...,None,None,315.117880,4.314566e+03,"POLYGON ((-75.27542 39.96896, -75.27542 39.968...","POLYGON ((-75.27542 39.96896, -75.27541 39.968...","POLYGON ((-75.27571 39.96864, -75.27582 39.968...",POINT (-75.27612 39.96913),POINT (-5712590.186 8483121.334),"POLYGON ((-5712589.472 8483026.846, -5712589.5..."
471,242023,Fee,Fee,LOC,CITY,Upper Darby Township,LOC,CITY,Upper Darby Township,LP,...,None,None,458.291779,1.188739e+04,"POLYGON ((-75.25664 39.94598, -75.25669 39.945...","POLYGON ((-75.25669 39.94597, -75.25671 39.945...","POLYGON ((-75.25671 39.94612, -75.25686 39.947...",POINT (-75.25609 39.94665),POINT (-5715384.150 8479507.849),"POLYGON ((-5715513.534 8479540.584, -5715516.4..."
472,242035,Fee,Fee,LOC,CITY,UPPER DARBY TOWNSHIP,LOC,CITY,UPPER DARBY TOWNSHIP,LP,...,None,None,552.520121,1.592591e+04,"POLYGON ((-75.27789 39.96684, -75.27789 39.966...","POLYGON ((-75.27789 39.96685, -75.27788 39.966...","POLYGON ((-75.27647 39.96684, -75.27650 39.966...",POINT (-75.27714 39.96644),POINT (-5713059.443 8483108.295),"POLYGON ((-5713024.922 8483221.016, -5713022.5..."


In [255]:
# Check intersecting road nodes.
parks_within_range = gpd.GeoDataFrame(parks_within_range, geometry = 'geom buffer diff', crs = 4326)

park_to_road = gpd.sjoin(parks_within_range, road_nodes.reset_index(), how = 'inner')
park_to_road = park_to_road.reset_index()
park_to_road['park_lon'] = park_to_road['geometry_m'].x
park_to_road['park_lat'] = park_to_road['geometry_m'].y
park_to_road

,index,OBJECTID,FeatClass,Category,Own_Type,Own_Name,Loc_Own,Mang_Type,Mang_Name,Loc_Mang,...,y,x,street_cou,lon,lat,highway,ref,geometry_m,park_lon,park_lat
0,0,81288,Fee,Fee,LOC,CNTY,Montgomery County,LOC,CNTY,Montgomery County,...,4.434577e+06,478109.735334,3,-75.256676,40.061161,None,None,POINT (-5697154.725 8485453.169),-5.697155e+06,8.485453e+06
1,1,82731,Fee,Fee,LOC,CNTY,Philadelphia County,LOC,CNTY,Philadelphia County,...,4.418573e+06,478915.650275,3,-75.246707,39.916993,None,None,POINT (-5719745.782 8476829.721),-5.719746e+06,8.476830e+06
2,1,82731,Fee,Fee,LOC,CNTY,Philadelphia County,LOC,CNTY,Philadelphia County,...,4.418590e+06,478940.035961,3,-75.246422,39.917142,crossing,None,POINT (-5719710.628 8476802.417),-5.719711e+06,8.476802e+06
3,1,82731,Fee,Fee,LOC,CNTY,Philadelphia County,LOC,CNTY,Philadelphia County,...,4.418596e+06,478948.607303,4,-75.246322,39.917196,traffic_signals,None,POINT (-5719698.056 8476792.890),-5.719698e+06,8.476793e+06
4,1,82731,Fee,Fee,LOC,CNTY,Philadelphia County,LOC,CNTY,Philadelphia County,...,4.418598e+06,478935.459380,3,-75.246476,39.917212,None,None,POINT (-5719701.652 8476812.613),-5.719702e+06,8.476813e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3813,472,242035,Fee,Fee,LOC,CITY,UPPER DARBY TOWNSHIP,LOC,CITY,UPPER DARBY TOWNSHIP,...,4.424118e+06,476299.210585,3,-75.277524,39.966878,None,None,POINT (-5713004.185 8483178.234),-5.713004e+06,8.483178e+06
3814,473,242038,Fee,Fee,LOC,CITY,UPPER DARBY TOWNSHIP,LOC,CITY,UPPER DARBY TOWNSHIP,...,4.423531e+06,476586.279558,3,-75.274141,39.961600,None,None,POINT (-5713712.736 8482492.281),-5.713713e+06,8.482492e+06
3815,473,242038,Fee,Fee,LOC,CITY,UPPER DARBY TOWNSHIP,LOC,CITY,UPPER DARBY TOWNSHIP,...,4.423569e+06,476683.632252,3,-75.273003,39.961942,None,None,POINT (-5713613.274 8482370.035),-5.713613e+06,8.482370e+06
3816,473,242038,Fee,Fee,LOC,CITY,UPPER DARBY TOWNSHIP,LOC,CITY,UPPER DARBY TOWNSHIP,...,4.423629e+06,476537.556647,3,-75.274715,39.962481,None,None,POINT (-5713594.849 8482607.942),-5.713595e+06,8.482608e+06


In [256]:
park_to_road[park_to_road['index'] == 453]

,index,OBJECTID,FeatClass,Category,Own_Type,Own_Name,Loc_Own,Mang_Type,Mang_Name,Loc_Mang,...,y,x,street_cou,lon,lat,highway,ref,geometry_m,park_lon,park_lat
3702,453,209646,Fee,Fee,LOC,CITY,City of Philadelphia,LOC,CITY,City of Philadelphia,...,4.429273e+06,493150.989575,4,-75.080253,40.013627,None,None,POINT (-5697726.578 8461422.644),-5.697727e+06,8.461423e+06
3703,453,209646,Fee,Fee,LOC,CITY,City of Philadelphia,LOC,CITY,City of Philadelphia,...,4.429296e+06,493169.700669,3,-75.080034,40.013838,None,None,POINT (-5697684.279 8461406.828),-5.697684e+06,8.461407e+06
3704,453,209646,Fee,Fee,LOC,CITY,City of Philadelphia,LOC,CITY,City of Philadelphia,...,4.429317e+06,493092.570321,3,-75.080938,40.014029,None,None,POINT (-5697690.013 8461527.279),-5.697690e+06,8.461527e+06


In [257]:
PtR = pd.merge(park_to_road, road_nodes['geometry'], left_on = 'osmid', right_index = True)
PtR = gpd.GeoDataFrame(PtR,geometry = 'geometry_y', crs = 4326)
PtR['geometry_m'] = PtR['geometry_y'].to_crs(3043)
PtR['Park_No'] = PtR['index']
PtR

,index,OBJECTID,FeatClass,Category,Own_Type,Own_Name,Loc_Own,Mang_Type,Mang_Name,Loc_Mang,...,street_cou,lon,lat,highway,ref,geometry_m,park_lon,park_lat,geometry_y,Park_No
0,0,81288,Fee,Fee,LOC,CNTY,Montgomery County,LOC,CNTY,Montgomery County,...,3,-75.256676,40.061161,None,None,POINT (-5697154.725 8485453.169),-5.697155e+06,8.485453e+06,POINT (-75.25668 40.06116),0
1,1,82731,Fee,Fee,LOC,CNTY,Philadelphia County,LOC,CNTY,Philadelphia County,...,3,-75.246707,39.916993,None,None,POINT (-5719745.782 8476829.721),-5.719746e+06,8.476830e+06,POINT (-75.24671 39.91699),1
2,1,82731,Fee,Fee,LOC,CNTY,Philadelphia County,LOC,CNTY,Philadelphia County,...,3,-75.246422,39.917142,crossing,None,POINT (-5719710.628 8476802.417),-5.719711e+06,8.476802e+06,POINT (-75.24642 39.91714),1
3,1,82731,Fee,Fee,LOC,CNTY,Philadelphia County,LOC,CNTY,Philadelphia County,...,4,-75.246322,39.917196,traffic_signals,None,POINT (-5719698.056 8476792.890),-5.719698e+06,8.476793e+06,POINT (-75.24632 39.91720),1
4,1,82731,Fee,Fee,LOC,CNTY,Philadelphia County,LOC,CNTY,Philadelphia County,...,3,-75.246476,39.917212,None,None,POINT (-5719701.652 8476812.613),-5.719702e+06,8.476813e+06,POINT (-75.24648 39.91721),1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3813,472,242035,Fee,Fee,LOC,CITY,UPPER DARBY TOWNSHIP,LOC,CITY,UPPER DARBY TOWNSHIP,...,3,-75.277524,39.966878,None,None,POINT (-5713004.185 8483178.234),-5.713004e+06,8.483178e+06,POINT (-75.27752 39.96688),472
3814,473,242038,Fee,Fee,LOC,CITY,UPPER DARBY TOWNSHIP,LOC,CITY,UPPER DARBY TOWNSHIP,...,3,-75.274141,39.961600,None,None,POINT (-5713712.736 8482492.281),-5.713713e+06,8.482492e+06,POINT (-75.27414 39.96160),473
3815,473,242038,Fee,Fee,LOC,CITY,UPPER DARBY TOWNSHIP,LOC,CITY,UPPER DARBY TOWNSHIP,...,3,-75.273003,39.961942,None,None,POINT (-5713613.274 8482370.035),-5.713613e+06,8.482370e+06,POINT (-75.27300 39.96194),473
3816,473,242038,Fee,Fee,LOC,CITY,UPPER DARBY TOWNSHIP,LOC,CITY,UPPER DARBY TOWNSHIP,...,3,-75.274715,39.962481,None,None,POINT (-5713594.849 8482607.942),-5.713595e+06,8.482608e+06,POINT (-75.27472 39.96248),473


In [258]:
# Check all parks within 1000m radius
start_time = time.time()
len1 = len(popgrid)
len2 = len(park_to_road)
len3 = int(np.ceil(len(park_to_road)/1000))
output = pd.DataFrame()
len_mat = 0
# Checking all the combinations at once is too performance intensive, it is broken down per 1000 (or what you want)
for i in range(len3):
    # Check all grid-park combinations per 1000
    l1, l2 = range(0,len1), range(i*1000,(i+1)*1000)
    listed = pd.DataFrame(list(product(l1, l2)))
    
    # Merge grid and park information
    grid_merged = pd.merge(listed, 
                           popgrid[['grid_id','centroid','centroid_m','grid_lon','grid_lat']], left_on = 0, right_index = True)
    node_merged = pd.merge(grid_merged, 
                           PtR[['Park_No','osmid','geometry_y','geometry_m','park_lon','park_lat']], left_on = 1, right_index = True)
    
    # Preset index for merging
    len4 = len(node_merged)
    node_merged['key'] = range(0,len(node_merged))
    node_merged = node_merged.set_index('key')
    node_merged = node_merged.loc[:, ~node_merged.columns.isin(['index'])]
    
    # Create lists for better computational performance
    glon = list(node_merged['grid_lon'])
    glat = list(node_merged['grid_lat'])
    plon = list(node_merged['park_lon'])
    plat = list(node_merged['park_lat'])
    
    # Get the euclidean distances
    mat = np.repeat(None,len(node_merged))
    for j in range(len(node_merged)):
        mat[j] = math.sqrt(abs(plon[j] - glon[j])**2 + abs(plat[j] - glat[j])**2)
    
    # Check if distances are within 1000m and join remaining info and concat in master df per 1000.
    mat_df = pd.DataFrame(mat)
    mat_df = mat_df[mat_df <=1000].dropna()
    mat_df = pd.DataFrame(mat_df)
    mat_df.columns = ['euclidean']    
    mat_df = mat_df.join(node_merged)
    
    output = pd.concat([output, mat_df])

    print((i+1),'/',len3,'done',round((time.time() - start_time) / 60,2),' mns')
    print('of',np.where(i+1 == len3, len2 % 1000 * 1000, len1*1000) ,'within 1000m:',len(mat_df))
    
    # Checks the number of the parks within 1000m.
    len_mat = len_mat + len(mat_df)
    
# Renaming columns
print('total combinations within distance',len_mat)
output.columns = ['euclidean','Grid_No','Park_entry_No','Grid_Id','Grid_coords_centroid','Grid_m_centroid','grid_lon','grid_lat',
                  'Park_No','Parkroad_osmid','Parkroad_coords_centroid','Parkroad_m_centroid','park_lon','park_lat']
output = output[['euclidean','Grid_No','Park_entry_No','Grid_Id','Grid_coords_centroid','Grid_m_centroid',
                  'Park_No','Parkroad_osmid','Parkroad_coords_centroid','Parkroad_m_centroid']]
output

1 / 4 done 0.25  mns
of 3891000 within 1000m: 13841
2 / 4 done 0.51  mns
of 3891000 within 1000m: 17826
3 / 4 done 0.76  mns
of 3891000 within 1000m: 18133
4 / 4 done 0.97  mns
of 818000 within 1000m: 13055
total combinations within distance 62855


,euclidean,Grid_No,Park_entry_No,Grid_Id,Grid_coords_centroid,Grid_m_centroid,Park_No,Parkroad_osmid,Parkroad_coords_centroid,Parkroad_m_centroid
867,403.623551,867,0,6117,POINT (-75.25369 40.06191),POINT (-5696918.007 8485126.249),0,1212727054,POINT (-75.25668 40.06116),POINT (-5697154.725 8485453.169)
7587,606.34087,3696,1,17765,POINT (-75.24471 39.92027),POINT (-5719144.331 8476752.871),1,7413577866,POINT (-75.24671 39.91699),POINT (-5719745.782 8476829.721)
7588,775.419116,3697,1,17766,POINT (-75.24247 39.92027),POINT (-5719055.281 8476476.898),1,7413577866,POINT (-75.24671 39.91699),POINT (-5719745.782 8476829.721)
7627,548.727076,3736,1,18050,POINT (-75.24247 39.91681),POINT (-5719607.250 8476298.769),1,7413577866,POINT (-75.24671 39.91699),POINT (-5719745.782 8476829.721)
7628,838.434156,3737,1,18051,POINT (-75.24022 39.91681),POINT (-5719518.170 8476022.773),1,7413577866,POINT (-75.24671 39.91699),POINT (-5719745.782 8476829.721)
...,...,...,...,...,...,...,...,...,...,...
3010463,913.539094,2720,3773,12225,POINT (-75.24920 39.98767),POINT (-5708569.997 8480770.261),464,3618217765,POINT (-75.24980 39.99311),POINT (-5707727.165 8481122.665)
3014318,982.48177,2684,3774,12085,POINT (-75.24471 39.98940),POINT (-5708117.175 8480308.183),464,111706635,POINT (-75.24989 39.99370),POINT (-5707635.357 8481164.408)
3084745,635.222698,3073,3792,13779,POINT (-75.26717 39.96867),POINT (-5712310.261 8481998.913),468,3425873704,POINT (-75.26671 39.96489),POINT (-5712894.247 8481748.976)
3088636,716.791621,3073,3793,13779,POINT (-75.26717 39.96867),POINT (-5712310.261 8481998.913),468,5544386939,POINT (-75.26643 39.96443),POINT (-5712957.523 8481690.948)


In [259]:
park_entry_centroid = gpd.GeoDataFrame(output[['euclidean','Grid_No','Park_No','Park_entry_No','Parkroad_osmid','Parkroad_m_centroid']],
                                      geometry = 'Parkroad_m_centroid', crs = 4326)
park_entry_centroid.to_file(r'C:\Users\bartb\Downloads\park_centroid.shp')

In [260]:
output[(output['Park_No'] == 231) & (output['Grid_No'] == 2639)]

,euclidean,Grid_No,Park_entry_No,Grid_Id,Grid_coords_centroid,Grid_m_centroid,Park_No,Parkroad_osmid,Parkroad_coords_centroid,Parkroad_m_centroid
3064856,214.025883,2639,1787,11857,POINT (-75.11895 39.99285),POINT (-5702575.446 8465078.414),231,109809486,POINT (-75.11806 39.99394),POINT (-5702367.869 8465026.271)
3068747,257.779648,2639,1788,11857,POINT (-75.11895 39.99285),POINT (-5702575.446 8465078.414),231,109999351,POINT (-75.11999 39.99417),POINT (-5702407.718 8465274.162)
3072638,208.087815,2639,1789,11857,POINT (-75.11895 39.99285),POINT (-5702575.446 8465078.414),231,5547581605,POINT (-75.11819 39.99395),POINT (-5702370.518 8465042.292)
3076529,202.093837,2639,1790,11857,POINT (-75.11895 39.99285),POINT (-5702575.446 8465078.414),231,2371556736,POINT (-75.11909 39.99406),POINT (-5702389.554 8465157.699)
3080420,315.122553,2639,1791,11857,POINT (-75.11895 39.99285),POINT (-5702575.446 8465078.414),231,109809494,POINT (-75.11790 39.99456),POINT (-5702262.806 8465038.940)
3084311,334.672172,2639,1792,11857,POINT (-75.11895 39.99285),POINT (-5702575.446 8465078.414),231,2371556737,POINT (-75.11788 39.99468),POINT (-5702242.736 8465042.228)
3088202,480.640883,2639,1793,11857,POINT (-75.11895 39.99285),POINT (-5702575.446 8465078.414),231,109899816,POINT (-75.11966 39.99567),POINT (-5702155.608 8465312.404)
3092093,456.479714,2639,1794,11857,POINT (-75.11895 39.99285),POINT (-5702575.446 8465078.414),231,109809504,POINT (-75.11773 39.99542),POINT (-5702119.247 8465062.415)


In [261]:
pd.DataFrame(output[(output['Park_No'] == 231) & (output['Grid_No'] == 2500)])


,euclidean,Grid_No,Park_entry_No,Grid_Id,Grid_coords_centroid,Grid_m_centroid,Park_No,Parkroad_osmid,Parkroad_coords_centroid,Parkroad_m_centroid
3064717,981.750582,2500,1787,11289,POINT (-75.11895 39.99976),POINT (-5701475.942 8465436.514),231,109809486,POINT (-75.11806 39.99394),POINT (-5702367.869 8465026.271)
3068608,945.81404,2500,1788,11289,POINT (-75.11895 39.99976),POINT (-5701475.942 8465436.514),231,109999351,POINT (-75.11999 39.99417),POINT (-5702407.718 8465274.162)
3072499,977.587613,2500,1789,11289,POINT (-75.11895 39.99976),POINT (-5701475.942 8465436.514),231,5547581605,POINT (-75.11819 39.99395),POINT (-5702370.518 8465042.292)
3076390,955.209924,2500,1790,11289,POINT (-75.11895 39.99976),POINT (-5701475.942 8465436.514),231,2371556736,POINT (-75.11909 39.99406),POINT (-5702389.554 8465157.699)
3080281,881.601233,2500,1791,11289,POINT (-75.11895 39.99976),POINT (-5701475.942 8465436.514),231,109809494,POINT (-75.11790 39.99456),POINT (-5702262.806 8465038.940)
3084172,862.227068,2500,1792,11289,POINT (-75.11895 39.99976),POINT (-5701475.942 8465436.514),231,2371556737,POINT (-75.11788 39.99468),POINT (-5702242.736 8465042.228)
3088063,690.904593,2500,1793,11289,POINT (-75.11895 39.99976),POINT (-5701475.942 8465436.514),231,109899816,POINT (-75.11966 39.99567),POINT (-5702155.608 8465312.404)
3091954,744.171919,2500,1794,11289,POINT (-75.11895 39.99976),POINT (-5701475.942 8465436.514),231,109809504,POINT (-75.11773 39.99542),POINT (-5702119.247 8465062.415)


In [262]:
start_time = time.time()

# Reinstate geographic elements
gp_entry = gpd.GeoDataFrame(output, geometry = 'Grid_coords_centroid', crs = 4326)
gp_entry['Grid_m_centroid'] = gpd.GeoSeries(gp_entry['Grid_m_centroid'], crs = 3043)
gp_entry['Parkroad_coords_centroid'] = gpd.GeoSeries(gp_entry['Parkroad_coords_centroid'], crs = 4326)
gp_entry['Parkroad_m_centroid'] = gpd.GeoSeries(gp_entry['Parkroad_m_centroid'], crs = 3043)

# Get index for merging later
roads = road_nodes.reset_index()

# Get the nearest entrance point for the grid centroids
mat2 = np.repeat(None,len(gp_entry))
for i in range(len(gp_entry)):
    nearest = int(roads['geometry'].sindex.nearest(gp_entry['Grid_coords_centroid'].iloc[i])[1])
    mat2[i] = roads['osmid'].iloc[nearest]
    if i % 100000 == 0: print(round(i/len(gp_entry)*100,1),'% done', round((time.time() - start_time) / 60,2),' mns')
print(len(mat2[mat2 != None]))

# Add all the information and the distance from the nearest point to the grid centroid in a geodataframe
pd.DataFrame(mat2)
gp_entry['grid_osm'] = mat2
gp_entry = pd.merge(gp_entry, road_nodes['geometry'], left_on = 'grid_osm', right_index = True)
gp_entry = gp_entry.loc[:, ~gp_entry.columns.isin(['grid_lon', 'grid_lat', 'park_lon', 'park_lat'])]
gp_entry['grid_entry_dist'] = round(gp_entry['Grid_m_centroid'].distance(gp_entry['geometry'].to_crs(3043)),3)
gp_entry = gp_entry.reset_index()
gp_entry

0.0 % done 0.0  mns
62855


,index,euclidean,Grid_No,Park_entry_No,Grid_Id,Grid_coords_centroid,Grid_m_centroid,Park_No,Parkroad_osmid,Parkroad_coords_centroid,Parkroad_m_centroid,grid_osm,geometry,grid_entry_dist
0,867,403.623551,867,0,6117,POINT (-75.25369 40.06191),POINT (-5696918.007 8485126.249),0,1212727054,POINT (-75.25668 40.06116),POINT (-5697154.725 8485453.169),110169822,POINT (-75.25279 40.06154),131.545
1,7587,606.34087,3696,1,17765,POINT (-75.24471 39.92027),POINT (-5719144.331 8476752.871),1,7413577866,POINT (-75.24671 39.91699),POINT (-5719745.782 8476829.721),110315111,POINT (-75.24450 39.92073),82.270
2,11478,568.460464,3696,2,17765,POINT (-75.24471 39.92027),POINT (-5719144.331 8476752.871),1,7342342352,POINT (-75.24642 39.91714),POINT (-5719710.628 8476802.417),110315111,POINT (-75.24450 39.92073),82.270
3,15369,555.169566,3696,3,17765,POINT (-75.24471 39.92027),POINT (-5719144.331 8476752.871),1,110326156,POINT (-75.24632 39.91720),POINT (-5719698.056 8476792.890),110315111,POINT (-75.24450 39.92073),82.270
4,19260,560.514359,3696,4,17765,POINT (-75.24471 39.92027),POINT (-5719144.331 8476752.871),1,9244049867,POINT (-75.24648 39.91721),POINT (-5719701.652 8476812.613),110315111,POINT (-75.24450 39.92073),82.270
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62850,3010462,934.702623,2719,3773,12224,POINT (-75.25145 39.98767),POINT (-5708658.692 8481045.677),464,3618217765,POINT (-75.24980 39.99311),POINT (-5707727.165 8481122.665),5543309000,POINT (-75.25127 39.98756),29.708
62851,3010463,913.539094,2720,3773,12225,POINT (-75.24920 39.98767),POINT (-5708569.997 8480770.261),464,3618217765,POINT (-75.24980 39.99311),POINT (-5707727.165 8481122.665),3971851907,POINT (-75.24913 39.98748),34.051
62852,3084745,635.222698,3073,3792,13779,POINT (-75.26717 39.96867),POINT (-5712310.261 8481998.913),468,3425873704,POINT (-75.26671 39.96489),POINT (-5712894.247 8481748.976),5543859996,POINT (-75.26566 39.96813),213.993
62853,3088636,716.791621,3073,3793,13779,POINT (-75.26717 39.96867),POINT (-5712310.261 8481998.913),468,5544386939,POINT (-75.26643 39.96443),POINT (-5712957.523 8481690.948),5543859996,POINT (-75.26566 39.96813),213.993


In [263]:
# Get only necessary road connections columns for network performance
road_conn = road_connections[['osmid','key','length','geometry']]
road_conn = road_conn.set_index('key')

In [264]:
#
gp_entry['Grid_m_centroid'].distance(gp_entry['Parkroad_m_centroid'].to_crs(3043))

0        403.623551
1        606.340870
2        568.460464
3        555.169566
4        560.514359
            ...    
62850    934.702623
62851    913.539094
62852    635.222698
62853    716.791621
62854    692.747670
Length: 62855, dtype: float64

In [265]:
import warnings
warnings.filterwarnings("ignore")

start_time = time.time()

parknode = list(gp_entry['Parkroad_osmid'])
gridnode = list(gp_entry['grid_osm'])

s_mat = list([])
s_mat1 = list([])
s_mat2 = list([])
s_mat3 = list([])
s_mat4 = list([])

for i in range(len(gp_entry)):
    # First try from population grid to park Dijkstra algorithm
    try:
        shortest = nx.shortest_path(graph, parknode[i], gridnode[i], 'travel_dist', method = 'dijkstra')
        s_mat.append(shortest)
        shortest_to = list(shortest[1:len(shortest)])
        shortest_to.append(0)
        s_mat1.append(shortest_to)
        s_mat2.append(list(np.repeat(i, len(shortest))))
        s_mat3.append(list(np.arange(0, len(shortest))))
        s_mat4.append('normal way')
        
    except:
        try:
            # Check the reverse
            shortest = nx.shortest_path(graph, gridnode[i], parknode[i], 'travel_dist', method = 'dijkstra')
            s_mat.append(shortest)
            shortest_to = list(shortest[1:len(shortest)])
            shortest_to.append(0)
            s_mat1.append(shortest_to)
            s_mat2.append(list(np.repeat(i, len(shortest))))
            s_mat3.append(list(np.arange(0, len(shortest))))
            s_mat4.append('reverse way')
        except:
            try:
                # Get the route to the nearest park node
                new_node = pd.DataFrame((abs(gp_entry['Parkroad_coords_centroid'].x[123381] - PtR['geometry_y'].x)**2
                                         + abs(gp_entry['Parkroad_coords_centroid'].y[123381] - PtR['geometry_y'].y)**2
                                        )**(1/2)).join(PtR['osmid']).sort_values(0).iloc[1,1]
                shortest = nx.shortest_path(graph, gridnode[i], new_node, 'travel_dist', method = 'dijkstra')
                s_mat.append(shortest)
                shortest_to = list(shortest[1:len(shortest)])
                shortest_to.append(0)
                s_mat1.append(shortest_to)
                s_mat2.append(list(np.repeat(i, len(shortest))))
                s_mat3.append(list(np.arange(0, len(shortest))))
                s_mat4.append('nearest way')
            except:
                # Print if none of the above three options gives any result
                print('index',i,'no route possible between osmid',gridnode[i],'and',parknode[i])
                pass
            
    if i % 10000 == 0: print(round(i/len(gp_entry)*100,2),'% done', round((time.time() - start_time) / 60,2),' mns')
print('100 % pathfinding done', round((time.time() - start_time) / 60,2),' mns')


0.0 % done 0.0  mns
15.91 % done 0.22  mns
31.82 % done 0.45  mns
47.73 % done 0.66  mns
63.64 % done 0.85  mns
79.55 % done 1.03  mns
95.46 % done 1.2  mns
100 % pathfinding done 1.26  mns


In [266]:
start_time = time.time()
# Unpack lists and add them in a dataframe
s_mat_u = [i for b in map(lambda x:[x] if not isinstance(x, list) else x, s_mat) for i in b]
s_mat_u1 = [i for b in map(lambda x:[x] if not isinstance(x, list) else x, s_mat1) for i in b]
s_mat_u2 = [i for b in map(lambda x:[x] if not isinstance(x, list) else x, s_mat2) for i in b]
s_mat_u3 = [i for b in map(lambda x:[x] if not isinstance(x, list) else x, s_mat3) for i in b]

routes = pd.DataFrame([s_mat_u,s_mat_u1,s_mat_u2,s_mat_u3]).transpose()

# Format df
routes.columns = ['from','to','route','step']
mat_key = list([])
for i in range(len(routes)):
    mat_key.append(str(int(s_mat_u[i])) + '-' + str(int(s_mat_u1[i])))
routes['key'] = mat_key
routes = routes.set_index('key')

# Add route information
routes = routes.join(road_conn, how = 'left')
routes = gpd.GeoDataFrame(routes, geometry = 'geometry', crs = 4326)
routes.sort_values(by = ['route','step'])
print('time taken', round((time.time() - start_time) / 60,2), 'mns')
routes

time taken 0.54 mns


,from,to,route,step,osmid,length,geometry
key,,,,,,,
1015355888-349355143,1015355888,349355143,2770,21,"[87325581, 87325582, 812998799]",63.696,"LINESTRING (-75.15103 39.94777, -75.15098 39.9..."
1015355888-349355143,1015355888,349355143,2774,23,"[87325581, 87325582, 812998799]",63.696,"LINESTRING (-75.15103 39.94777, -75.15098 39.9..."
1015355888-349355143,1015355888,349355143,2775,22,"[87325581, 87325582, 812998799]",63.696,"LINESTRING (-75.15103 39.94777, -75.15098 39.9..."
1015355888-349355143,1015355888,349355143,2776,21,"[87325581, 87325582, 812998799]",63.696,"LINESTRING (-75.15103 39.94777, -75.15098 39.9..."
1015355888-349355143,1015355888,349355143,2777,25,"[87325581, 87325582, 812998799]",63.696,"LINESTRING (-75.15103 39.94777, -75.15098 39.9..."
...,...,...,...,...,...,...,...
989146009-4696697785,989146009,4696697785,39859,8,344740382,2.457,"LINESTRING (-75.18702 39.95520, -75.18705 39.9..."
989146009-4696697785,989146009,4696697785,39863,5,344740382,2.457,"LINESTRING (-75.18702 39.95520, -75.18705 39.9..."
989146009-4696697785,989146009,4696697785,39866,3,344740382,2.457,"LINESTRING (-75.18702 39.95520, -75.18705 39.9..."


In [267]:
start_time = time.time()
# get single (dissolved) line per route, attach information.
routes2 = routes[['route','geometry']].dissolve('route')
routes2['way_calculated'] = s_mat4
routes2['route_cost'] = routes.groupby('route')['length'].sum()
routes2['num_steps'] = routes.groupby('route')['step'].max()
routes2.index = routes2.index.astype(int)
routes2 = pd.merge(routes2, gp_entry[['Grid_No','Park_No','Park_entry_No','grid_entry_dist','Parkroad_osmid','grid_osm']],
                   left_index = True, right_index = True)
routes2['total_cost'] = routes2['route_cost'] + routes2['grid_entry_dist']
routes2['gridpark_no'] = routes2['Grid_No'].astype(str) +'-'+ routes2['Park_No'].astype(str)
print('time taken', round((time.time() - start_time) / 60,2), 'mns')
routes2 

time taken 0.33 mns


,geometry,way_calculated,route_cost,num_steps,Grid_No,Park_No,Park_entry_No,grid_entry_dist,Parkroad_osmid,grid_osm,total_cost,gridpark_no
route,,,,,,,,,,,,
0,"MULTILINESTRING ((-75.25279 40.06154, -75.2536...",normal way,381.109,2,867,0,0,131.545,1212727054,110169822,512.654,867-0
1,"MULTILINESTRING ((-75.24621 39.92077, -75.2446...",normal way,685.560,7,3696,1,1,82.270,7413577866,110315111,767.830,3696-1
2,"MULTILINESTRING ((-75.24621 39.92077, -75.2446...",normal way,607.859,6,3696,1,2,82.270,7342342352,110315111,690.129,3696-1
3,"MULTILINESTRING ((-75.24621 39.92077, -75.2446...",normal way,597.428,5,3696,1,3,82.270,110326156,110315111,679.698,3696-1
4,"MULTILINESTRING ((-75.24621 39.92077, -75.2446...",normal way,599.316,5,3696,1,4,82.270,9244049867,110315111,681.586,3696-1
...,...,...,...,...,...,...,...,...,...,...,...,...
62850,"MULTILINESTRING ((-75.24987 39.98844, -75.2499...",normal way,1092.424,17,2719,464,3773,29.708,3618217765,5543309000,1122.132,2719-464
62851,"MULTILINESTRING ((-75.24987 39.98844, -75.2499...",normal way,754.742,13,2720,464,3773,34.051,3618217765,3971851907,788.793,2720-464
62852,"MULTILINESTRING ((-75.26768 39.96453, -75.2674...",normal way,672.249,10,3073,468,3792,213.993,3425873704,5543859996,886.242,3073-468


In [268]:
start_time = time.time()
# Get the entry point of the park that is closest to the grid centroid
gridpark_score = routes2.iloc[routes2.groupby('gridpark_no')['total_cost'].idxmin()]

# Check if the route to the closest entry point is within 1000m.
gridpark_score = gridpark_score[gridpark_score['total_cost'] <= 1000]

# The score is the inverse of the total cost (if a park is 100m (on route) away its score is 900 (1000 - 100))
gridpark_score['score'] = 1000 - gridpark_score['total_cost']

# Get the population weighted score
gridpark_score = pd.merge(gridpark_score, popgrid[['PoP2015_Number','geometry']],
                          left_on = 'Grid_No', right_index = True, how = 'outer')

gridpark_score['score'] = gridpark_score['score'].fillna(0)
gridpark_score['population_score'] = gridpark_score['score'] * gridpark_score['PoP2015_Number']

gridpark_score['Park_No'] = gridpark_score['Park_No'].fillna(-1)
gridpark_score['Park_No'] = gridpark_score['Park_No'].astype(int)

gridpark_score['Park_entry_No'] = gridpark_score['Park_No'].fillna(-1)
gridpark_score['Park_entry_No'] = gridpark_score['Park_No'].astype(int)

gridpark_score.index = gridpark_score.index.fillna(-1)
gridpark_score.index = gridpark_score.index.astype(int)

gridpark_score['num_steps'] = gridpark_score['num_steps'].fillna(-1)
gridpark_score['num_steps'] = gridpark_score['num_steps'].astype(int)

print('time taken', round((time.time() - start_time) / 60,2), 'mns')

gridpark_score

time taken 0.02 mns


,geometry_x,way_calculated,route_cost,num_steps,Grid_No,Park_No,Park_entry_No,grid_entry_dist,Parkroad_osmid,grid_osm,total_cost,gridpark_no,score,PoP2015_Number,geometry_y,population_score
56477,"MULTILINESTRING ((-75.00833 40.12855, -75.0087...",normal way,649.034,5,10,349,349,47.215,1.100560e+08,109892511,696.249,10-349,303.751,127,"POLYGON ((-75.00778 40.13004, -75.00778 40.128...",38576.377
10970,"MULTILINESTRING ((-75.00396 40.05987, -75.0041...",normal way,537.833,7,1000,45,45,20.765,1.099861e+08,7826688686,558.598,1000-45,441.402,187,"POLYGON ((-74.99880 40.05932, -74.99880 40.057...",82542.174
5694,"MULTILINESTRING ((-74.99301 40.05945, -74.9929...",normal way,833.041,11,1001,200,200,58.594,5.184435e+09,110425022,891.635,1001-200,108.365,57,"POLYGON ((-74.99431 40.05932, -74.99431 40.057...",6176.805
5685,"MULTILINESTRING ((-74.99466 40.05787, -74.9945...",normal way,415.485,4,1001,35,35,58.594,1.104250e+08,110425022,474.079,1001-35,525.921,57,"POLYGON ((-74.99431 40.05932, -74.99431 40.057...",29977.497
5688,"MULTILINESTRING ((-74.99568 40.05885, -74.9956...",normal way,217.754,2,1001,45,45,58.594,1.101575e+08,110425022,276.348,1001-45,723.652,57,"POLYGON ((-74.99431 40.05932, -74.99431 40.057...",41248.164
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
-1,None,NaN,NaN,-1,3886,-1,-1,NaN,NaN,NaN,NaN,NaN,0.000,5,"POLYGON ((-75.25706 39.88308, -75.25706 39.881...",0.000
-1,None,NaN,NaN,-1,3887,-1,-1,NaN,NaN,NaN,NaN,NaN,0.000,39,"POLYGON ((-75.25257 39.88308, -75.25257 39.881...",0.000
-1,None,NaN,NaN,-1,3888,-1,-1,NaN,NaN,NaN,NaN,NaN,0.000,5,"POLYGON ((-75.25033 39.88308, -75.25033 39.881...",0.000
-1,None,NaN,NaN,-1,3889,-1,-1,NaN,NaN,NaN,NaN,NaN,0.000,3,"POLYGON ((-75.25931 39.88135, -75.25931 39.879...",0.000


In [ ]:
gridpark_lines[['geometry_x','total_cost','Grid_No','Park_No','PoP2015_Number','population_score']].to_file(
    r'C:\Users\bartb\Downloads\gridpark_lines.shp')

gridpark_nodes[['geometry_y','total_cost','Grid_No','Park_No','PoP2015_Number','population_score']].to_file(
    r'C:\Users\bartb\Downloads\gridpark_nodes.shp')

In [269]:
start_time = time.time()
# Group the park scores per grid.
grid_score = pd.DataFrame(gridpark_score.groupby('Grid_No')['score'].sum().sort_values(ascending=False))
grid_score['population_score (mln)'] = gridpark_score.groupby('Grid_No')['population_score'].sum()/1000000
grid_score['count'] = gridpark_score.groupby('Grid_No')['score'].count()

grid_score = grid_score.join(popgrid[['geometry','PoP2015_Number']])
grid_score['lines'] = gpd.GeoDataFrame(gridpark_score[['Grid_No','geometry_x']], geometry = 'geometry_x', crs = 4326).dissolve('Grid_No')

# The merge contains grids with no access to parks within 1000m, these scores are set to 0
grid_score = gpd.GeoDataFrame(grid_score, geometry = 'geometry', crs = 4326)

print('time taken', round((time.time() - start_time) / 60,2), 'mns')
grid_score

time taken 0.03 mns


,score,population_score (mln),count,geometry,PoP2015_Number,lines
Grid_No,,,,,,
3008,5351.464,2.092422,10,"POLYGON ((-75.15151 39.97471, -75.15151 39.972...",391,"(LINESTRING (-75.154257 39.972587, -75.1537682..."
3507,5295.312,3.246026,13,"POLYGON ((-75.15825 39.94015, -75.15825 39.938...",613,"(LINESTRING (-75.159796 39.939227, -75.1594865..."
2972,4936.451,5.079608,10,"POLYGON ((-75.15151 39.97644, -75.15151 39.974...",1029,"(LINESTRING (-75.1522929 39.97578079999999, -7..."
2931,4809.817,6.459584,13,"POLYGON ((-75.15600 39.97817, -75.15600 39.976...",1343,"(LINESTRING (-75.1586216 39.97446099999999, -7..."
3007,4771.753,2.791476,10,"POLYGON ((-75.15376 39.97471, -75.15376 39.972...",585,"(LINESTRING (-75.154257 39.972587, -75.1537682..."
...,...,...,...,...,...,...
1249,0.000,0.000000,1,"POLYGON ((-75.23236 40.04897, -75.23236 40.047...",199,None
1248,0.000,0.000000,1,"POLYGON ((-75.23460 40.04897, -75.23460 40.047...",137,None
1247,0.000,0.000000,1,"POLYGON ((-75.24134 40.04897, -75.24134 40.047...",137,None


In [270]:
grid_score['population_score (mln)'].mean()

0.3016954020904652

In [271]:
start_time = time.time()
# group the grid scores per park (which parks will be popular)
park_score = pd.DataFrame(gridpark_score.groupby('Park_No')['score'].sum().sort_values(ascending=False))
park_score['population_score (mln.)'] = gridpark_score.groupby('Park_No')['population_score'].sum()/1000000
park_score['Park access to pop.'] = gridpark_score.groupby('Park_No')['PoP2015_Number'].sum()
park_score['count'] = gridpark_score.groupby('Park_No')['score'].count()
park_score = park_score.join(parks_within_range[['geometry','OBJECTID']], how = 'outer')
park_score = park_score[park_score.index >= 0]
park_score['OBJECTID'] = park_score['OBJECTID'].astype(int)
print('time taken', round((time.time() - start_time) / 60,2), 'mns')
park_score['lines'] = gpd.GeoDataFrame(gridpark_score[['Park_No','geometry_x']], geometry = 'geometry_x', crs = 4326).dissolve('Park_No')
park_score

time taken 0.0 mns


,score,population_score (mln.),Park access to pop.,count,geometry,OBJECTID,lines
0,487.346,0.001462,3.0,1.0,"MULTIPOLYGON (((-75.24324 40.05599, -75.24326 ...",81288,"(LINESTRING (-75.25279 40.061537, -75.25368000..."
1,2070.047,0.575987,2059.0,7.0,"POLYGON ((-75.24706 39.91693, -75.24708 39.916...",82731,"(LINESTRING (-75.2462141 39.92077040000001, -7..."
2,3648.994,1.565927,4482.0,10.0,"MULTIPOLYGON (((-75.23911 39.94226, -75.23909 ...",82731,"(LINESTRING (-75.2431787 39.929475399999994, -..."
3,8308.990,2.058810,6499.0,25.0,"POLYGON ((-75.14993 39.96159, -75.15020 39.961...",1260,"(LINESTRING (-75.149658 39.96234099999999, -75..."
4,5708.769,1.378378,5052.0,18.0,"MULTIPOLYGON (((-75.17209 40.03406, -75.17261 ...",1348,"(LINESTRING (-75.171826 40.034192, -75.1715073..."
...,...,...,...,...,...,...,...
469,NaN,NaN,NaN,NaN,"POLYGON ((-75.25807 39.94941, -75.25801 39.949...",242021,None
470,NaN,NaN,NaN,NaN,"POLYGON ((-75.27571 39.96864, -75.27582 39.968...",242022,None
471,NaN,NaN,NaN,NaN,"POLYGON ((-75.25671 39.94612, -75.25686 39.947...",242023,None
472,NaN,NaN,NaN,NaN,"POLYGON ((-75.27647 39.96684, -75.27650 39.966...",242035,None


In [ ]:
park_lines = gpd.GeoDataFrame(park_score[['score', 'population_score (mln.)','count', 'lines']], 
                              geometry = 'lines', crs = 4326)
park_lines.to_file(r'C:\Users\bartb\Downloads\park_entrance_routes.shp')

park_s = gpd.GeoDataFrame(park_score[['score', 'population_score (mln.)','count', 'geometry']], 
                          geometry = 'geometry', crs = 4326)
park_s.to_file(r'C:\Users\bartb\Downloads\park_score_entrance.shp')

In [ ]:
grid_lines = gpd.GeoDataFrame(grid_score[['score', 'population_score (mln)','count','PoP2015_Number','lines']], 
                              geometry = 'lines', crs = 4326)
grid_lines.to_file(r'C:\Users\bartb\Downloads\grid_entrance_routes.shp')

grid_s = gpd.GeoDataFrame(grid_score[['score', 'population_score (mln)','count','PoP2015_Number','geometry']], 
                          geometry = 'geometry', crs = 4326)
grid_s.to_file(r'C:\Users\bartb\Downloads\grid_score_entrance.shp')

In [272]:
print(round((time.time() - start) / 60,2),'mns')

7.91 mns
